In [9]:
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from selenium.webdriver.remote import webelement
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
from IPython.display import Image, display
import os
import sys
from time import sleep

import pandas as pd
import gspread
from google.auth import default
from google.auth.exceptions import RefreshError

In [10]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)
driver.get("https://www.google.com")
# driver.execute_script("window.open('https://nowsecure.nl/', '_blank')")
# time.sleep(15)
# driver.switch_to.window(driver.window_handles[1])

# driver.switch_to.frame(0)

# driver.find_element(By.XPATH, '//*[@id="challenge-stage"]/div/label/input').click()


In [11]:
def login_indeed_google(driver, google_email, google_password):
    """
    Function to log in to Indeed via Google login.
    
    Parameters:
    driver (webdriver): The Selenium WebDriver instance.
    google_email (str): The Google email address for the Indeed account.
    google_password (str): The Google account password.
    """
    driver.get("https://secure.indeed.com/account/login")
    
    try:
        # Wait for the Google login button and click it
        google_login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login-google-button"))
        )
        google_login_button.click()

        # Switch to Google login window
        main_window = driver.current_window_handle
        WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
        for handle in driver.window_handles:
            if handle != main_window:
                driver.switch_to.window(handle)
                break

        # Enter email and submit
        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "identifierId"))
        )
        email_input.send_keys(google_email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)

        # Enter password and submit
        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "Passwd"))
        )
        password_input.send_keys(google_password)
        password_input.send_keys(Keys.RETURN)

        # Wait for Google authentication to complete
        WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(1))
        driver.switch_to.window(main_window)

        # Click "Continue" button if it exists
        try:
            continue_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-INsAgc"))
            )
            continue_button.click()
        except TimeoutException:
            print("Nút 'Continue' không tồn tại hoặc không xuất hiện.")

        # Verify login success on Indeed
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "user-account-link"))
        )
        print("Logged in successfully via Google!")

    except TimeoutException as e:
        print(f"Error: Timeout occurred - {e}")
    except NoSuchElementException as e:
        print(f"Error: Element not found - {e}")
    except Exception as e:
        print(f"Error during login via Google: {e}")


In [12]:
username = "chidinhm3@gmail.com"
password = "dinh0911"
login_indeed_google(driver, username, password)


KeyboardInterrupt: 

In [42]:
def get_all_jobs(driver, url):
    # Switch to main window and navigate to the given URL
    driver.switch_to.window(driver.window_handles[0])
    driver.get(url)

    try:
        # Wait for the job list to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//li[contains(@class, 'css-1ac2h1w')]//h2[contains(@class, 'jobTitle')]//span"))
        )
    except TimeoutException:
        print("Timeout: Trang không tải xong hoặc không tìm thấy danh sách công việc.")
        return []

    all_jobs_data = []

    # Locate all job elements
    jobs = driver.find_elements(By.XPATH, "//li[contains(@class, 'css-1ac2h1w')]//h2[contains(@class, 'jobTitle')]//span")

    for i, job in enumerate(jobs):
        try:
            print(f"\nProcessing Job {i + 1}: {job.text}")

            # Scroll the job element into view and wait for it to be clickable
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", job)
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable(job))

            try:
                job.click()  # Attempt to click the job
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", job)  # Fallback: Click using JavaScript

            time.sleep(2)  # Allow time for the job details to load

            # Collect job details
            link = driver.current_url

            # Job Title
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h2[data-testid='simpler-jobTitle']").text
            except Exception:
                print("Không tìm thấy tiêu đề công việc.")
                title = "Không xác định"

            # Company Name
            try:
                company = driver.find_element(By.CSS_SELECTOR, "a.jobsearch-JobInfoHeader-companyNameLink").text
            except Exception:
                print("Không tìm thấy tên công ty.")
                company = "Không xác định"

            # Location
            try:
                location = driver.find_element(By.CSS_SELECTOR, "div[data-testid='jobsearch-JobInfoHeader-companyLocation']").text
            except Exception:
                print("Không tìm thấy địa điểm.")
                location = "Không xác định"

            # Salary (if available)
            try:
                salary = driver.find_element(By.CSS_SELECTOR, "#jobDetailsSection span.js-match-insights-provider-4pmm6z").text
            except Exception:
                print("Không tìm thấy mức lương.")
                salary = "Không hiển thị"

            # Append all job details to the list
            all_jobs_data.append([link, company, title,salary, location])
            print(f"Job {i + 1} details captured successfully.")

        except StaleElementReferenceException:
            print(f"StaleElementReferenceException: Không thể thao tác với công việc thứ {i + 1}.")
        except ElementClickInterceptedException:
            print(f"ElementClickInterceptedException: Click bị chặn tại công việc thứ {i + 1}.")
        except Exception as e:
            print(f"Lỗi không xác định khi xử lý công việc thứ {i + 1}: {e}")

    return all_jobs_data


In [24]:
import gspread
from google.oauth2.service_account import Credentials
try:
    # Đường dẫn đến tệp JSON của bạn
    SERVICE_ACCOUNT_FILE = 'snappy-charge-444008-u3-2d010cb44a06.json'

    # Phạm vi quyền truy cập
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # Xác thực với Google Sheets API
    credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(credentials)

    # Mở Google Sheet bằng URL
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1FUAXm33xCAOQtlJgvn7h6jqCXDPYH9AuFBpVOOB3AnY/edit?usp=sharing'
    sheet = gc.open_by_url(spreadsheet_url)

    # Lấy trang tính đầu tiên
    work_sheet = sheet.get_worksheet(0)

    # Lấy hàng tiêu đề
    header = work_sheet.row_values(1)

    # Kiểm tra sự tồn tại của các tiêu đề cần thiết
    required_columns = ["Company name", "Title job", "Location job", "Salary"]
    for column in required_columns:
        if column not in header:
            print(f"Column '{column}' not found in the header.")
            exit(1)

    # Tìm vị trí các cột theo tiêu đề
    company_col = header.index("Company name") + 1  # Cột "Company name"
    job_col = header.index("Title job") + 1         # Cột "Title job"
    locate_col = header.index("Location job") + 1  # Cột "Location job"
    salary_col = header.index("Salary") + 1        # Cột "Salary"

    # Kiểm tra kết nối bằng cách in tiêu đề
    print(f"Successfully accessed sheet: {sheet.title}")
    print("Headers:", header)

    # Đọc tất cả dữ liệu
    data = work_sheet.get_all_records()
    print("Dữ liệu từ Google Sheet:", data)

except Exception as e:
    print(f"Error: {str(e)}")


Successfully accessed sheet: Crawling info job Indeed
Headers: ['Link job', 'Company name', 'Title job', 'Salary', 'Location job']
Dữ liệu từ Google Sheet: [{'Link job': 'https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64', 'Company name': '', 'Title job': '', 'Salary': '', 'Location job': ''}]


In [35]:
import gspread
from google.oauth2.service_account import Credentials

def update_google_sheet(sheet, data):
    """
    Cập nhật Google Sheet với dữ liệu công việc đã thu thập.

    Args:
        sheet: Đối tượng Google Sheet.
        data: Danh sách các dữ liệu công việc để thêm.
    """
    try:
        # Kiểm tra xem dữ liệu có rỗng không
        if not data:
            print("Không có dữ liệu để cập nhật.")
            return

        # Chuẩn bị dữ liệu dưới dạng danh sách các danh sách
        # Mỗi danh sách con tương ứng với một hàng dữ liệu
        # Giả sử data là danh sách các danh sách hoặc danh sách các từ điển
        # Nếu data là danh sách các từ điển, chuyển đổi nó thành danh sách các danh sách
        if isinstance(data[0], dict):
            # Xác định thứ tự cột theo tiêu đề đã lấy trước đó
            headers = sheet.row_values(1)
            prepared_data = []
            for row in data:
                prepared_row = [
                    row.get("Link job", ""),
                    row.get("Company name", ""),
                    row.get("Title job", ""),
                    row.get("Salary", ""),
                    row.get("Location job", "")
                ]
                prepared_data.append(prepared_row)
        else:
            # Nếu data đã là danh sách các danh sách
            prepared_data = data

        # Sử dụng append_rows để thêm nhiều hàng cùng một lúc
        sheet.append_rows(prepared_data, value_input_option='RAW')
        print(f"Đã thêm thành công {len(prepared_data)} hàng vào Google Sheet.")

    except Exception as e:
        print(f"Lỗi khi cập nhật Google Sheet: {str(e)}")


In [43]:
link = "https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64"
jobs = get_all_jobs(driver, link)
update_google_sheet(work_sheet, jobs)


Processing Job 1: Senior Associate, Restructuring
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 1 details captured successfully.

Processing Job 2: Associate Portfolio Manager / Associate Investment Advisor
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 2 details captured successfully.

Processing Job 3: Associate Portfolio Manager, Quantitative Equity
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 3 details captured successfully.

Processing Job 4: Regional Director, Mid
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 4 details captured successfully.

Processing Job 5: 2025 Insurance, Summer Sales Support (4-16 months)
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 5 details captured successfully.

Processing Job 6: Senior Buy Side Equity Analyst
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 6 details captured successfully.

Processing Job 7: Directeur régional, Marché I

In [29]:
link_col_index = header.index("Link job") + 1  # Index bắt đầu từ 0 nên cần +1
company_col_index = header.index("Company name") + 1  # Index bắt đầu từ 0 nên cần +1

# Lấy toàn bộ dữ liệu từ cột "Link" và "Company"
link_column_data = work_sheet.col_values(link_col_index)
company_column_data = work_sheet.col_values(company_col_index)
print(len(link_column_data))
# Duyệt qua và chỉ lấy các giá trị LinkedIn từ cột "Link" khi cột "Company" trống
linkedin_links = [
    (i + 1, link_column_data[i])
    for i in range(1, len(link_column_data))  # Bắt đầu từ 1 để bỏ qua header
]
print(linkedin_links)
# In ra số thứ tự và link hợp lệ
for row_number, link in linkedin_links:
    print(f"Row {row_number}: {link}")

2
[(2, 'https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64')]
Row 2: https://ca.indeed.com/jobs?q=CFA&l=Vancouver%2C+BC&vjk=2f3201c73baa7f64


In [32]:
for i in linkedin_links:
    try:
        # Lấy dữ liệu công việc từ liên kết
        profile_data = get_all_jobs(driver, i[1])

        # Cập nhật Google Sheet nếu có dữ liệu
        if profile_data:
            update_google_sheet(sheet, profile_data, i[0])
        else:
            print(f"No data found for Row {i[0]}, skipping update.")

        print(f"Data for Row {i[0]}: {profile_data}")

    except Exception as e:
        print(f"Error processing Row {i[0]}: {e}")

# Đóng trình duyệt và thông báo hoàn thành
# browser.quit()
print("Crawling and Google Sheet update completed successfully!")



Processing Job 1: Senior Associate, Restructuring
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 1 details captured successfully.

Processing Job 2: Associate Portfolio Manager / Associate Investment Advisor
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 2 details captured successfully.

Processing Job 3: Associate Portfolio Manager, Quantitative Equity
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 3 details captured successfully.

Processing Job 4: 2025 Insurance, Summer Sales Support (4-16 months)
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 4 details captured successfully.

Processing Job 5: Regional Director, Mid
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 5 details captured successfully.

Processing Job 6: Directeur régional, Marché Intermédiaire – Vancouver
Không tìm thấy tiêu đề công việc.
Không tìm thấy tên công ty.
Job 6 details captured successfully.

Processing Job 7: Portfo